<a href="https://colab.research.google.com/github/mcjackson10/BeauxSGProject/blob/main/All_in_One_Dog_Detector_Python_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# All-in-One Dog Detector and Deterrent Script
#
# This script uses a Raspberry Pi to:
# 1. Capture video from a Pi Camera.
# 2. Use a TensorFlow Lite model to detect objects in the video feed.
# 3. If a 'dog' is detected with sufficient confidence, it triggers actions.
# 4. Sends a notification to your phone via Pushover.
# 5. Controls a servo and a relay via GPIO pins to aim and operate a squirt gun.

import cv2
import numpy as np
import time
import requests
from picamera2 import Picamera2
from gpiozero import Servo, OutputDevice
from tflite_runtime.interpreter import Interpreter

# --- User-Defined Settings & Credentials ---

# Pushover Notification Credentials
# IMPORTANT: Replace these with your actual Pushover User Key and API Token/Key
PUSHOVER_USER_KEY = "ug5on3hcov7y64ioemekzk8i7p7y35"
PUSHOVER_API_TOKEN = "arxfj6m4owmuwu37x1m3tm6rqciza7"

# Model and Label File Paths
MODEL_PATH = "model.tflite"
LABEL_PATH = "labelmap.txt"

# Detection Thresholds
CONFIDENCE_THRESHOLD = 0.60  # Minimum confidence score (60%) to consider a detection valid
TARGET_LABEL = 'dog'         # The object we are looking for

# Cooldown Period
# Time in seconds to wait before sending another notification to prevent spam
NOTIFICATION_COOLDOWN = 30
last_notification_time = 0

# --- GPIO and Hardware Setup ---

# Initialize GPIO components using gpiozero
# GPIO pins are specified using BCM numbering
try:
    # Servo for aiming (connected to GPIO18, which is Pin 12)
    # min_pulse_width and max_pulse_width may need tuning for your specific servo
    servo = Servo(18, min_pulse_width=0.5/1000, max_pulse_width=2.5/1000)

    # Relay for the water pump (connected to GPIO17, which is Pin 11)
    # initial_value=False means the relay is off by default
    relay = OutputDevice(17, active_high=True, initial_value=False)

    # Set initial servo position to center
    servo.mid()
    print("GPIO components initialized successfully.")

except Exception as e:
    print(f"Error initializing GPIO components: {e}")
    print("Running in simulation mode. GPIO actions will be printed to the console.")
    servo = None
    relay = None

# --- Function Definitions ---

def send_pushover_notification(message):
    """Sends a notification message using the Pushover service."""
    if PUSHOVER_USER_KEY == "Yug5on3hcov7y64ioemekzk8i7p7y35" or PUSHOVER_API_TOKEN == "arxfj6m4owmuwu37x1m3tm6rqciza7":
        print("Pushover credentials not set. Skipping notification.")
        return

    try:
        payload = {
            "token": PUSHOVER_API_TOKEN,
            "user": PUSHOVER_USER_KEY,
            "message": message,
            "title": "Dog Alert!"
        }
        r = requests.post("https://api.pushover.net/1/messages.json", data=payload, timeout=5)
        r.raise_for_status() # Raise an exception for bad status codes
        print("Pushover notification sent successfully.")
    except requests.exceptions.RequestException as e:
        print(f"Failed to send Pushover notification: {e}")

def squirt_sequence():
    """Aims the servo and activates the pump for a short duration."""
    print("Executing squirt sequence...")
    if servo and relay:
        # Example: Aim slightly left, squirt, aim slightly right, squirt, return to center
        servo.value = -0.5  # Corresponds to a position left of center
        time.sleep(0.5)
        relay.on()
        time.sleep(0.5) # Squirt duration
        relay.off()

        time.sleep(0.5)

        servo.value = 0.5   # Corresponds to a position right of center
        time.sleep(0.5)
        relay.on()
        time.sleep(0.5) # Squirt duration
        relay.off()

        time.sleep(0.5)
        servo.mid() # Return to center
    else:
        print("[SIMULATED] Squirt sequence activated.")

def load_labels(path):
    """Loads the labels file."""
    with open(path, 'r') as f:
        return [line.strip() for line in f.readlines()]

# --- Main Program ---

# Load labels and initialize TFLite interpreter
labels = load_labels(LABEL_PATH)
interpreter = Interpreter(MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

# Initialize the Pi Camera
picam2 = Picamera2()
picam2.configure(picam2.create_preview_configuration(main={"format": 'RGB888', "size": (640, 480)}))
picam2.start()
print("Camera initialized. Starting detection loop...")

# Main detection loop
try:
    while True:
        # Capture a frame from the camera
        frame = picam2.capture_array()

        # Prepare the frame for the model
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        im_resized = cv2.resize(image_rgb, (width, height))
        input_data = np.expand_dims(im_resized, axis=0)

        # Perform detection by running the model with the frame as input
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()

        # Retrieve detection results
        boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Bounding box coordinates
        classes = interpreter.get_tensor(output_details[3]['index'])[0] # Class index of detected objects
        scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects

        # Loop over all detections and process them
        for i in range(len(scores)):
            current_score = scores[i]
            current_class_id = int(classes[i])
            current_label = labels[current_class_id]

            if current_score > CONFIDENCE_THRESHOLD and current_label == TARGET_LABEL:
                # Check if cooldown period has passed
                current_time = time.time()
                if (current_time - last_notification_time) > NOTIFICATION_COOLDOWN:
                    print(f"--- DOG DETECTED with {current_score:.2%} confidence! ---")

                    # Send notification
                    send_pushover_notification("Dog detected in the kitchen!")

                    # (Optional) Trigger the deterrent
                    squirt_sequence()

                    # Update the notification timestamp
                    last_notification_time = current_time

                # Draw bounding box and label on the frame for visual feedback
                ymin, xmin, ymax, xmax = boxes[i]
                (left, right, top, bottom) = (xmin * 640, xmax * 640, ymin * 480, ymax * 480)
                cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (0, 255, 0), 2)
                label_text = f"{current_label}: {current_score:.2%}"
                cv2.putText(frame, label_text, (int(left), int(top) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the resulting frame (optional, useful for debugging)
        cv2.imshow('Object Detector', frame)

        # Press 'q' to quit
        if cv2.waitKey(1) == ord('q'):
            break

finally:
    # Clean up resources
    cv2.destroyAllWindows()
    picam2.stop()
    if servo and relay:
        servo.close()
        relay.close()
    print("\nProgram terminated gracefully.")